In [1]:
# Because MobileNet has version that is based out of TensorFlow 1
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

!git clone --quiet https://github.com/tensorflow/models.git

TensorFlow 1.x selected.
1.15.2


In [2]:
# Install the TF Object Detection API (TFOD)
%%bash
pip install --upgrade pip
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1600108 sha256=370dd379372b1feaa874e7603932d5adb35ba859cd242943ffc495f82cd86b3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0y6hbkup/wheels/16/63/fd/1293066bf448f757979af1b45c59b0e33b46f12d4454d12fde
Successfully built object-detection


# Create tflite graph

Normal model:

In [3]:
checkpoint_link = 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz'
folder_name = 'ssd_mobilenet_v2_coco_2018_03_29'

!rm -rf folder_name
!wget -q $checkpoint_link -O checkpoints.tar.gz
!tar -xvf checkpoints.tar.gz
!rm checkpoints.tar.gz

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_model/variables/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.index
ssd_mobilenet_v2_coco_2018_03_29/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.data-00000-of-00001


In [4]:
#Export a TFLite compatible model graph

# First: https://stackoverflow.com/questions/49880939/tf-object-detection-api-detection-model-retraining-object-detection-protos-ssd/49883742#49883742
# So remove from /content/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config the line containing batch_norm_trainable.

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$folder_name/pipeline.config \
    --trained_checkpoint_prefix=$folder_name/model.ckpt \
    --output_directory=$folder_name \
    --add_postprocessing_op=true
print("======Graph generated========")
!ls -lh $folder_name/*.pb

Instructions for updating:
Please use `layer.__call__` method instead.
W1125 15:54:20.585149 139775181379456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1125 15:54:23.161205 139775181379456 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1125 15:54:23.196407 139775181379456 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1125 15:54:23.230549 139775181379456 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

QAT model:

In [5]:
checkpoint_link = 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz'
folder_name = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03'

!rm -rf folder_name
!wget -q $checkpoint_link -O checkpoints.tar.gz
!tar -xvf checkpoints.tar.gz
!rm checkpoints.tar.gz


#Export a TFLite compatible model graph

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$folder_name/pipeline.config \
    --trained_checkpoint_prefix=$folder_name/model.ckpt \
    --output_directory=$folder_name \
    --add_postprocessing_op=true
print("======Graph generated========")
!ls -lh $folder_name/*.pb

ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt.data-00000-of-00001
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt.index
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt.meta
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/pipeline.config
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/tflite_graph.pb
ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/tflite_graph.pbtxt


# Quantization

In [1]:
#Initial setup
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

'2.3.0'

Normal model:

In [2]:
#Select model to quantize
import os

folder_name = 'ssd_mobilenet_v2_coco_2018_03_29'
model_to_be_quantized = folder_name + '/tflite_graph.pb'
if os.path.getsize(model_to_be_quantized) > 0:
    print(f"We are quantizing: {model_to_be_quantized}")
else:
    print("Model not found!")

We are quantizing: ssd_mobilenet_v2_coco_2018_03_29/tflite_graph.pb


In [ ]:
# Representative dataset:
!wget -q https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/releases/download/v0.9.0/train_samples_coco.tar.gz
!tar -xvf train_samples_coco.tar.gz
!rm train_samples_coco.tar.gz

rep_ds = tf.data.Dataset.list_files("train_samples/*.jpg")
HEIGHT, WIDTH = 300, 300

def representative_dataset_gen():
    for image_path in rep_ds:
        img = tf.io.read_file(image_path)
        img = tf.io.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        resized_img = tf.image.resize(img, (HEIGHT, WIDTH))
        resized_img = resized_img[tf.newaxis, :]
        yield [resized_img]

In [20]:
#Quantize and convert
quant_targets = ["dr", "fp16", "int8_fallback", "normal"]

for quantization_strategy in quant_targets:
  converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
      graph_def_file=model_to_be_quantized, 
      input_arrays=['normalized_input_image_tensor'],
      output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'],
      input_shapes={'normalized_input_image_tensor': [1, 300, 300, 3]}
  )
  converter.allow_custom_ops = True

  if quantization_strategy=="dr":
      converter.optimizations = [tf.lite.Optimize.DEFAULT]
  elif quantization_strategy=="fp16":
      converter.target_spec.supported_types = [tf.float16]
      converter.optimizations = [tf.lite.Optimize.DEFAULT]
  elif quantization_strategy=="int8_fallback":  # Modified for the tf website version... https://www.tensorflow.org/lite/performance/post_training_integer_quant
      converter.optimizations = [tf.lite.Optimize.DEFAULT]
      converter.representative_dataset = representative_dataset_gen
      converter.quantized_input_stats = {"normalized_input_image_tensor": (128, 128)}
  else:
      pass
  tflite_model = converter.convert()

  tflite_filename = 'mobilenet' + "_" + quantization_strategy + ".tflite"
  open(tflite_filename, 'wb').write(tflite_model)
  print(f"TFLite model generated with {quantization_strategy}")


!ls -lh $tflite_filename

TFLite model generated with dr
TFLite model generated with fp16
TFLite model generated with int8_fallback
TFLite model generated with normal
-rw-r--r-- 1 root root 65M Nov 25 16:14 mobilenet_normal.tflite


QAT model:

In [23]:
#Select model to quantize
import os

model_checkpoint_name = "ssd_mobilenet_v2_quantized_coco_fp32"
model_to_be_quantized = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/tflite_graph.pb'
if os.path.getsize(model_to_be_quantized) > 0:
    print(f"We are quantizing: {model_checkpoint_name}")
else:
    print("Model not found!")

We are quantizing: ssd_mobilenet_v2_quantized_coco_fp32


In [25]:
#Quantize and serialize (`uint8` are WIP)
original_precision = "fp32"
quantization_strategy = "int8"

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=model_to_be_quantized, 
    input_arrays=['normalized_input_image_tensor'],
    output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'],
    input_shapes={'normalized_input_image_tensor': [1, 300, 300, 3]}
)
converter.allow_custom_ops = True

assert model_checkpoint_name.split("_")[-1]==original_precision, "Mismatch between the precisions."
assert quantization_strategy == "int8", "Only integer precision is allowed for this model."
# https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692063937
converter.inference_input_type = tf.uint8
converter.quantized_input_stats = {"normalized_input_image_tensor": (128, 128)}
# No additional optimization: https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692545579

tflite_model = converter.convert()

tflite_filename = 'mobilenet' + "_" + quantization_strategy + ".tflite"
open(tflite_filename, 'wb').write(tflite_model)
print(f"TFLite model generated with {quantization_strategy}")
!ls -lh $tflite_filename

TFLite model generated with int8
-rw-r--r-- 1 root root 6.0M Nov 25 16:30 mobilenet_int8.tflite


In [ ]:
#Install the latest edge TPU compiler
%%bash
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu-compiler

In [27]:
#Compile the file
!edgetpu_compiler -s /content/mobilenet_int8.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 1249 ms.

Input model: /content/mobilenet_int8.tflite
Input size: 5.93MiB
Output model: mobilenet_int8_edgetpu.tflite
Output size: 6.59MiB
On-chip memory used for caching model parameters: 6.35MiB
On-chip memory remaining for caching model parameters: 1.39MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 111
Operation log: mobilenet_int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 108
Number of operations that will run on CPU: 3

Operator                       Count      Status

CONCATENATION                  2          Mapped to Edge

In [28]:
#Download models
from google.colab import files
import glob
for file in glob.glob("./*.tflite"):
  files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>